<a href="https://colab.research.google.com/github/kissone418/1090828-1/blob/master/1140318%E5%8D%9A%E5%85%8B%E4%BE%86%E6%9F%A5%E8%A9%A2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import requests
from bs4 import BeautifulSoup
import csv

# 設定 HTTP 請求標頭，模擬瀏覽器行為
HEADERS = {
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_4) AppleWebKit/537.36 "
                  "(KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36"
}

CSV_FILE = 'output.csv'

# 初始化 CSV，確保標題正確
def init_csv():
    try:
        with open(CSV_FILE, 'r', encoding='utf-8-sig') as f:
            pass  # 如果文件已存在則不做任何動作
    except FileNotFoundError:
        with open(CSV_FILE, 'w', newline='', encoding='utf-8-sig') as f:
            writer = csv.writer(f)
            writer.writerow(['排序', '書名', '作者', '出版社', '出版地', 'ISBN', '採購數量', '定價'])

# 爬取書籍資訊
def fetch_book_info(url):
    try:
        response = requests.get(url, headers=HEADERS)
        response.raise_for_status()  # 檢查 HTTP 回應碼
    except requests.RequestException as e:
        print(f"無法取得網頁內容，錯誤: {e}")
        return None

    soup = BeautifulSoup(response.text, 'html.parser')

    try:
        book_title = soup.find("title").text.strip()

        # 獲取 meta 資訊
        meta_tags = soup.find_all("meta")
        meta_content = str(meta_tags[3])
        meta_list = meta_content.split("，")

        author = next((s[3:] for s in meta_list if "作者：" in s), "未知")
        publisher = next((s[4:] for s in meta_list if "出版社：" in s), "未知")
        isbn = next((s[5:] for s in meta_list if "ISBN：" in s), "未知")

        # 獲取定價
        price_tag = soup.find("em")
        price = price_tag.string if price_tag else "未知"

        return [book_title, author, publisher, '', isbn, '', price]
    except Exception as e:
        print(f"解析書籍資訊時發生錯誤: {e}")
        return None

# 寫入 CSV，使用 `utf-8-sig` 以便 Excel 正確顯示中文
def write_to_csv(data):
    if data:
        with open(CSV_FILE, 'a', newline='', encoding='utf-8-sig') as f:
            writer = csv.writer(f)
            writer.writerow(['A'] + data)
        print("✅ 書籍資訊已存入 CSV 檔案。")

# 主程式
def main():
    init_csv()
    print("📚 請到博客來查詢一本書，並複製其網址到此程式。")
    print("輸入 'Q' 退出。")

    while True:
        url = input("請輸入博客來網址: ")
        if url.strip().upper() == "Q":
            print("📌 程式結束。")
            break
        elif not url.startswith("https://www.books.com.tw/products/"):
            print("⚠️ 請輸入有效的博客來書籍網址！")
            continue

        book_info = fetch_book_info(url)
        if book_info:
            write_to_csv(book_info)

if __name__ == "__main__":
    main()


📚 請到博客來查詢一本書，並複製其網址到此程式。
輸入 'Q' 退出。
請輸入博客來網址: https://www.books.com.tw/products/0010977157?sloc=main
✅ 書籍資訊已存入 CSV 檔案。
請輸入博客來網址: https://www.books.com.tw/products/0010763075?loc=P_1_001
✅ 書籍資訊已存入 CSV 檔案。


KeyboardInterrupt: Interrupted by user